# 入门云原生AI - 1. 从mnist开始体验

在这个示例中，我们将演示：

* 下载并准备数据
* 利用Arena提交单机训练任务,并且查看训练任务状态和日志，并通过TensorBoard查看训练任务
* 为您的训练结果部署一个模型预测的在线服务。
* 在Notebook中调用您的在线服务，验证模型准确率。

### 1. 下载TensorFlow样例源代码到 /root/models 目录

In [2]:
! mkdir  -p /root/models
! git clone https://code.aliyun.com/xiaozhou/tensorflow-sample-code.git /root/models/tensorflow-sample-code

mkdir: cannot create directory '/root/models': File exists
fatal: destination path '/root/models/tensorflow-sample-code' already exists and is not an empty directory.


### 2. 下载mnist数据到 /root/dataset/mnist

In [3]:
! mkdir -p /root/output
! mkdir -p /root/dataset/mnist && \
  cd /root/dataset/mnist && \
  curl -O https://code.aliyun.com/xiaozhou/tensorflow-sample-code/raw/master/data/t10k-images-idx3-ubyte.gz && \
  curl -O https://code.aliyun.com/xiaozhou/tensorflow-sample-code/raw/master/data/t10k-labels-idx1-ubyte.gz && \
  curl -O https://code.aliyun.com/xiaozhou/tensorflow-sample-code/raw/master/data/train-images-idx3-ubyte.gz && \
  curl -O https://code.aliyun.com/xiaozhou/tensorflow-sample-code/raw/master/data/train-labels-idx1-ubyte.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1610k    0 1610k    0     0  5225k      0 --:--:-- --:--:-- --:--:-- 5228k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4542    0  4542    0     0  22307      0 --:--:-- --:--:-- --:--:-- 22374
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9680k    0 9680k    0     0  25.2M      0 --:--:-- --:--:-- --:--:-- 25.2M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 28881    0 28881    0     0   134k      0 --:--:-- --:--:-- --:--:--  135k


### 3. 查看目录结构, 其中`dataset`是数据目录，`models`是模型代码目录，`output`是训练结果目录。

In [5]:
! tree -I ai-starter -L 3 /root

/root
|-- dataset
|   `-- mnist
|       |-- t10k-images-idx3-ubyte.gz
|       |-- t10k-labels-idx1-ubyte.gz
|       |-- train-images-idx3-ubyte.gz
|       `-- train-labels-idx1-ubyte.gz
|-- models
|   `-- tensorflow-sample-code
|       |-- README.md
|       |-- data
|       |-- mnist-tf
|       |-- models
|       |-- mpijob
|       `-- tfjob
|-- output
|-- public

25 directories, 12 files


### 4.检查可用GPU资源

In [6]:
! arena top node

NAME                                IPADDRESS      ROLE    GPU(Total)  GPU(Allocated)
cn-shanghai.i-uf60zgmfu7nvxxxxxxxx  192.168.0.46   <none>  1           0
cn-shanghai.i-uf6hf2g6lwyvxxxxxxxx  192.168.0.203  <none>  0           0
-----------------------------------------------------------------------------------------
Allocated/Total GPUs In Cluster:
0/1 (0%)  


### 5.通过Arena提交训练任务

5.1 可以根据您的需要设置JOB_NAME，建议在多人共同使用的时候，设置自己独有的JOB_NAME

5.2 这里 `$USER_DATA_NAME` 代表您的Notebook 使用的共享存储，存储的根目录和Notebook中`/root`对应。 由集群管理员在部署工作环境时指定[部署数据科学家工作环境（Notebook）](../docs/setup/SETUP_NOTEBOOK.md)创建。 
* `$USER_DATA_NAME` 是存放您私有数据的共享存储，文件内容对应Notebook的 `/root` 目录
* `$PUBLIC_DATA_NAME` 是存放公共数据的共享存储，文件内容对应Notebook的 `/root/public` 目录。在arena的命令中，如果需要使用公共存储里的文件，可以指定参数 `--data=$PUBLIC_DATA_NAME:/public`，并在训练代码中指定容器使用`/public`目录里的代码或数据

在下面的提交命令中：
* `--data=$USER_DATA_NAME:/training` 表示将共享存储映射到训练任务的`/training`目录。
* `/training`目录下的子目录`/training/models/tensorflow-sample-code` 就是步骤1拷贝源代码的位置
* `/training`目录下的子目录`/training/dataset/mnist`就是步骤2下载数据的位置
* `/training`目录下的子目录`/training/output`就是步骤3创建的训练结果输出的位置。

In [54]:
# Set the Job Name
%env JOB_NAME=tf-mnist
%env USER_DATA_NAME=training-data
# Submit a training job 
# using code and data from Data Volume
!arena submit tf \
             --name=$JOB_NAME \
             --gpus=1 \
             --data=$USER_DATA_NAME:/training \
             --tensorboard \
             --image=tensorflow/tensorflow:1.11.0-gpu-py3 \
             --logdir=/training/output/mnist \
             "python /training/models/tensorflow-sample-code/tfjob/docker/mnist/main.py --max_steps 5000 --data_dir /training/dataset/mnist --log_dir /training/output/mnist"

env: JOB_NAME=tf-mnist
configmap/tf-mnist-tfjob created
configmap/tf-mnist-tfjob labeled
service/tf-mnist-tensorboard created
deployment.extensions/tf-mnist-tensorboard created
tfjob.kubeflow.org/tf-mnist created
INFO[0008] The Job tf-mnist has been submitted successfully 
INFO[0008] You can run `arena get tf-mnist --type tfjob` to check the job status 


> - `Arena`命令的`--logdir`指定`tensorboard`从训练任务的指定目录读取event  
> - 完整参数可以参考[命令行文档](https://github.com/kubeflow/arena/blob/master/docs/cli/arena_submit_tfjob.md)

### 6.检查模型训练状态
当任务状态从`Pending`转为`Running`后就可以查看日志和GPU使用率了。这里`-e`为了方便检查任务`Pending`的原因。通常看到`[Pulling] pulling image "tensorflow/tensorflow:1.11.0-gpu-py3"`代表容器镜像过大，导致任务处于`Pending`。这时可以重复执行下列命令直到任务状态变为`Running`。

In [55]:
! arena get $JOB_NAME -e

STATUS: RUNNING
NAMESPACE: default
TRAINING DURATION: 1s

NAME      STATUS   TRAINER  AGE  INSTANCE          NODE
tf-mnist  RUNNING  TFJOB    1s   tf-mnist-chief-0  192.168.0.31

Your tensorboard will be available on:
http://192.168.0.203:32116   

Events: 
No events for pending pod


### 7.实时检查日志
此时可以通过调整`--tail=`的数值展示输出的行数。默认为显示全部日志。
想要获取实时日志， 可以执行`-f`参数

In [56]:
! arena logs --tail=50 $JOB_NAME

2019-04-03T11:30:20.434833662Z WARNING:tensorflow:From /training/models/tensorflow-sample-code/tfjob/docker/mnist/main.py:41: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
2019-04-03T11:30:20.434891738Z Instructions for updating:
2019-04-03T11:30:20.434898475Z Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
2019-04-03T11:30:20.473209783Z WARNING:tensorflow:From /usr/local/lib/python3.5/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
2019-04-03T11:30:20.473235115Z Instructions for updating:
2019-04-03T11:30:20.473239615Z Please write your own downloading logic.
2019-04-03T11:30:20.483963934Z WARNING:tensorflow:From /usr/local/lib/python3.5/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extr

### 8. 查看实时训练的GPU使用情况

In [6]:
! arena top job $JOB_NAME

INSTANCE NAME     GPU(Device Index)  GPU(Duty Cycle)  GPU(Memory MiB)  STATUS     NODE
tf-mnist-chief-0  N/A                N/A              N/A              Succeeded  


### 9. 查看训练结果
#### 9.1 通过TensorBoard查看训练趋势
您可以使用 `192.168.1.117:30670` 访问 Tensorboard。如果您通过笔记本电脑无法直接访问 Tensorboard，可以考虑在您的笔记本电脑使用 `sshuttle`。例如：`sshuttle -r root@41.82.59.51 192.168.0.0/16`。其中`41.82.59.51`为集群内某个节点的外网IP，且该外网IP可以通过ssh访问。

In [7]:
# show job detail
! arena get $JOB_NAME

STATUS: SUCCEEDED
NAMESPACE: default
TRAINING DURATION: 13m

NAME      STATUS     TRAINER  AGE  INSTANCE          NODE
tf-mnist  SUCCEEDED  TFJOB    20m  tf-mnist-chief-0  N/A

Your tensorboard will be available on:
http://192.168.0.203:30532   


![](1-1-tensorboard.jpg)

#### 9.2. 查看模型训练产生的结果文件, 在`/root/output`下生成了训练结果.

`/root/output/mnist` 目录中是训练过程中产生的checkpoint文件，代表训练结束时模型的变量状态。

In [57]:
! tree -I ai-starter -L 3 /root/output

/root/output
|-- mnist
|   |-- checkpoint
|   |-- model.ckpt-4500.data-00000-of-00001
|   |-- model.ckpt-4500.index
|   |-- model.ckpt-4500.meta
|   |-- model.ckpt-4600.data-00000-of-00001
|   |-- model.ckpt-4600.index
|   |-- model.ckpt-4600.meta
|   |-- model.ckpt-4700.data-00000-of-00001
|   |-- model.ckpt-4700.index
|   |-- model.ckpt-4700.meta
|   |-- model.ckpt-4800.data-00000-of-00001
|   |-- model.ckpt-4800.index
|   |-- model.ckpt-4800.meta
|   |-- model.ckpt-4900.data-00000-of-00001
|   |-- model.ckpt-4900.index
|   |-- model.ckpt-4900.meta
|   |-- test
|   |   |-- events.out.tfevents.1554286572.tf-mnist-chief-0
|   |   `-- events.out.tfevents.1554291023.tf-mnist-chief-0
|   `-- train
|       |-- events.out.tfevents.1554286571.tf-mnist-chief-0
|       `-- events.out.tfevents.1554291021.tf-mnist-chief-0
`-- mnist-model
    `-- 1
        |-- saved_model.pb
        `-- variables

6 directories, 21 files


### 10. 将训练过程中产生的checkpoint转换为模型文件
checkpoint文件不能直接用于部署预测服务，需要进行一次模型导出，将checkpoint文件中的模型状态导出为可以被预测服务识别的模型文件。 我们可以通过arena 提交一个转换任务，声明将`output/mnist`目录下的checkpoint文件导出到`output/mnist-model`目录。

In [3]:
! arena submit tf \
     --name=export-model \
     --workers=1 \
     --gpus=1 \
     --data=$USER_DATA_NAME:/training \
     --image=tensorflow/tensorflow:1.11.0-gpu-py3 \
     "python /training/models/tensorflow-sample-code/tfjob/docker/mnist/export_model.py \
      --checkpoint_step=4900 \
     --checkpoint_path=/training/output/mnist /training/output/mnist-model/ "

configmap/export-model-tfjob created
configmap/export-model-tfjob labeled
tfjob.kubeflow.org/export-model created
INFO[0008] The Job export-model has been submitted successfully 
INFO[0008] You can run `arena get export-model --type tfjob` to check the job status 


#### 10.1.查看模型导出任务的执行状态

In [60]:
! arena get export-model

STATUS: FAILED
NAMESPACE: default
TRAINING DURATION: 6s

NAME          STATUS  TRAINER  AGE  INSTANCE              NODE
export-model  FAILED  TFJOB    1m   export-model-chief-0  N/A


#### 10.2 导出任务执行完毕后，可以在`output/mnist-model`目录中看到导出的模型文件。

In [62]:
! tree -I ai-starter -L 3 /root/output/mnist-model

/root/output/mnist-model
`-- 1
    |-- saved_model.pb
    `-- variables
        |-- variables.data-00000-of-00001
        `-- variables.index

2 directories, 3 files


### 11. 部署预测服务

我们得到可以用于部署预测服务的模型文件后，可以通过`arena serve` 提交一个模型预测的在线任务。 

* `--data=$USER_DATA_NAME:/training` 表示将共享存储目录挂载到预测服务的`/training`目录
* `--modelPath=/training/output/mnist-model` 表示预测服务使用的模型文件目录，就是我们在步骤12中导出的模型文件位置

In [63]:
! arena serve tensorflow \
    --servingName=mnist \
    --modelName=mnist \
    --image=tensorflow/serving:latest  \
    --data=$USER_DATA_NAME:/training \
    --modelPath=/training/output/mnist-model

configmap/mnist-tf-serving created
configmap/mnist-tf-serving labeled
configmap/mnist-tensorflow-serving-cm created
service/mnist-tensorflow-serving created
deployment.extensions/mnist-tensorflow-serving created


#### 11.2查看预测服务

我们可以查到到预测服务的部署状态，以及入口访问地址。

In [31]:
! arena serve list

NAME   TYPE        VERSION  STATUS   CLUSTER-IP
mnist  Tensorflow           RUNNING  172.19.82.216


### 12. 通过预测服务验证模型准确率

In [91]:
# 安装必要的python 库
! pip install requests

You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


#### 12.1 定义函数，函数内容通过Http调用预测服务

以下代码定义一个python方法`pick_image_and_predict`，这个方法会从mnist的测试数据集中随机选择一张图片，作为请求的参数，通过http方式调用预测服务，得到模型预测的记过。这个方法执行后会同时打印图片的真实值，和通过预测服务推理得到的值。 您可以用于判断模型的准确率是否满足要求。

In [92]:
import matplotlib.pyplot as plt
import numpy as np
import random
import requests
import json
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline
data_dir="/root/dataset/mnist/"
mnist = input_data.read_data_sets(data_dir, one_hot=True)
test_images = mnist.test.images
test_labels = mnist.test.labels
digits = ['0','1','2','3','4','5','6','7','8','9']
# -*- coding: utf-8 -*-

def show(idx, title):
  plt.figure()
  plt.imshow(test_images[idx].reshape(28,28))
  plt.axis('off')
  plt.title('\n\n{}'.format(title), fontdict={'size': 16})

def predict(url, num):
    test_cls = np.argmax(test_labels, axis=1)
    show(num, 'The Picture is {}'.format(test_cls[num]))
    headers = {"content-type": "application/json"}
    metadata=requests.get(url + '/metadata')
    data = json.dumps({"signature_name": "predict_images", "dropout/Placeholder": 1.0,"inputs": test_images[num].reshape(1, 784).tolist()})
    json_response = requests.post(model_api+':predict', data=data, headers=headers)
    scores = json.loads(json_response.text)['outputs']
    predicted_digits_idx = np.argmax(scores)
    print('预测识别的数字: {}'.format(digits[predicted_digits_idx]))
    return scores

def pick_image_and_predict(model_api):
    random_image = random.randint(0,len(test_images)-1)
    score = predict(model_api, random_image)

Extracting /root/dataset/mnist/train-images-idx3-ubyte.gz
Extracting /root/dataset/mnist/train-labels-idx1-ubyte.gz
Extracting /root/dataset/mnist/t10k-images-idx3-ubyte.gz
Extracting /root/dataset/mnist/t10k-labels-idx1-ubyte.gz


#### 12.2 调用预测服务
我们可以得到真实值和预测值。 您可以多执行几次或者修改代码增加执行次数，比较预测结果和真是结果，判断模型的预测准确率。

这里`mnist-tensorflow-serving` 代表预测服务的服务域名，您也可以改为IP地址，IP地址的获取在12步中，通过`arena serve list` 可以得到的预测服务的服务IP。

In [ ]:
model_api='http://mnist-tensorflow-serving:8501/v1/models/mnist'
pick_image_and_predict(model_api)

###  13. 删除已经完成的任务

In [88]:
# delete job
! arena delete $JOB_NAME
! arena delete export-model
# delete serving job
! arena serve delete mnist

service "tf-mnist-tensorboard" deleted
deployment.extensions "tf-mnist-tensorboard" deleted
tfjob.kubeflow.org "tf-mnist" deleted
configmap "tf-mnist-tfjob" deleted
INFO[0006] The Job tf-mnist has been deleted successfully 
tfjob.kubeflow.org "export-model" deleted
configmap "export-model-tfjob" deleted
INFO[0006] The Job export-model has been deleted successfully 
configmap "mnist-tensorflow-serving-cm" deleted
service "mnist-tensorflow-serving" deleted
deployment.extensions "mnist-tensorflow-serving" deleted
configmap "mnist-tf-serving" deleted
INFO[0002] The Serving job mnist has been deleted successfully 


恭喜！您已经使用 `arena` 成功运行了训练作业，而且还能轻松检查 Tensorboard。

总结，希望您通过本次演示了解：
1. 如何准备代码和数据，并将其放入数据卷中
2. 如何在训练任务中引用数据卷，并且使用其中的代码和数据
3. 如何利用arena管理您的训练任务。
4. 为您的训练结果部署一个模型预测的在线服务。
5. 在Notebook中调用您的在线服务，验证模型准确率。

以上是使用`Arena`在云上进行模型训练的例子，您可以通过修改代码`/root/models/tensorflow-sample-code/tfjob/docker/mnist/main.py`重新提交，实现迭代的模型开发目的。